## SWOT Level 3 Altimetry Data Processing

### Open the downloaded data

In [14]:
import xarray as xr
import matplotlib.pyplot as plt
import glob

In [13]:
!ls /home/jovyan/shared/cycle_500

SWOT_L3_LR_SSH_Basic_500_001_20230423T175831_20230423T184936_v1.0.nc
SWOT_L3_LR_SSH_Basic_500_002_20230423T184937_20230423T194041_v1.0.nc
SWOT_L3_LR_SSH_Basic_500_003_20230423T194042_20230423T203147_v1.0.nc
SWOT_L3_LR_SSH_Basic_500_004_20230423T203148_20230423T212253_v1.0.nc
SWOT_L3_LR_SSH_Basic_500_005_20230423T212253_20230423T221358_v1.0.nc
SWOT_L3_LR_SSH_Basic_500_006_20230423T221359_20230423T230504_v1.0.nc
SWOT_L3_LR_SSH_Basic_500_007_20230423T230505_20230423T235610_v1.0.nc
SWOT_L3_LR_SSH_Basic_500_008_20230423T235610_20230424T004715_v1.0.nc
SWOT_L3_LR_SSH_Basic_500_009_20230424T004716_20230424T013821_v1.0.nc
SWOT_L3_LR_SSH_Basic_500_010_20230424T013822_20230424T022926_v1.0.nc
SWOT_L3_LR_SSH_Basic_500_011_20230424T022927_20230424T032033_v1.0.nc
SWOT_L3_LR_SSH_Basic_500_012_20230424T032033_20230424T041138_v1.0.nc
SWOT_L3_LR_SSH_Basic_500_013_20230424T041139_20230424T050244_v1.0.nc
SWOT_L3_LR_SSH_Basic_500_014_20230424T050245_20230424T055350_v1.0.nc
SWOT_L3_LR_SSH_Basic_500_015_20230

In [16]:
data_dir = '/home/jovyan/shared/cycle_500/*'
data_files = glob.glob(data_dir)

### Visualize absolute dynamic topography for the first record

In [21]:
ds = xr.open_dataset(data_files[-2])
ds

<xarray.Dataset> Size: 27MB
Dimensions:         (num_lines: 9860, num_pixels: 69, num_nadir: 2132)
Coordinates:
    latitude        (num_lines, num_pixels) float64 5MB ...
    longitude       (num_lines, num_pixels) float64 5MB ...
Dimensions without coordinates: num_lines, num_pixels, num_nadir
Data variables:
    time            (num_lines) datetime64[ns] 79kB ...
    mdt             (num_lines, num_pixels) float64 5MB ...
    ssha            (num_lines, num_pixels) float64 5MB ...
    ssha_noiseless  (num_lines, num_pixels) float64 5MB ...
    i_num_line      (num_nadir) int16 4kB ...
    i_num_pixel     (num_nadir) int8 2kB ...
Attributes: (12/42)
    Conventions:                     CF-1.7
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Swath
    comment:                         Sea Surface Height measured by Altimetry
    geospatial_lat_units:            degrees_north
    geospatial_lon_units:            degrees_east
    ...                              ...
    geospatial_lat_min:              -78.272132
    geospatial_lat_max:              78.272321
    geospatial_lon_min:              106.057318
    geospatial_lon_max:              273.003289
    data_used:                       SWOT KaRIn L2_LR_SSH PGC0 (NASA/CNES). D...
    doi:                             https://doi.org/10.24400/527896/A01-2023...